In [3]:
# Import necessary libraries
import numpy as np
import xarray as xr
import pandas as pd
import matplotlib.pyplot as plt
import os

In [72]:
# Import necessary libraries
import numpy as np
import xarray as xr
import pandas as pd
import matplotlib.pyplot as plt
import os

# X data
fp = '/rc_scratch/reba1583/variable_yr_files_4/'

IWV = xr.open_mfdataset(fp+'IWV').isel(time = slice(0,25)).IWV
IWV = IWV.transpose('time', 'lon','lat_index').values
EFLUX = xr.open_mfdataset(fp+'EFLUX').isel(time = slice(0,25)).EFLUX
EFLUX = EFLUX.transpose('time', 'lon','lat_index').values
LWTNET = xr.open_mfdataset(fp+'LWTNET').isel(time = slice(0,25)).LWTNET
LWTNET = LWTNET.transpose('time', 'lon','lat_index').values
SF = xr.open_mfdataset(fp+'sf').isel(time = slice(0,25)).sf
SF = SF.transpose('time', 'lon','lat_index').values
SLP = xr.open_mfdataset(fp+'SLP').isel(time = slice(0,25)).SLP
SLP = SLP.transpose('time', 'lon','lat_index').values
T = xr.open_mfdataset(fp+'T').isel(time = slice(0,25)).T
T = T.transpose('time', 'lon','lat_index').values
U = xr.open_mfdataset(fp+'U').isel(time = slice(0,25)).U
U = U.transpose('time', 'lon','lat_index').values
V = xr.open_mfdataset(fp+'V').isel(time = slice(0,25)).V
V = V.transpose('time', 'lon','lat_index').values

data = np.stack([ IWV, EFLUX, LWTNET,
                 SF, SLP, T, U, V], 3)

del V 
del U
del T
del IWV
del EFLUX
del SF
del SLP
del LWTNET

# Y data
Y = xr.open_mfdataset('/rc_scratch/reba1583/Y_data/Y_AR_only.nc').isel(time = slice(0,25))
Y = Y.Y.transpose('time','lon','lat_index').values

# times for final xarray
variable_times = pd.to_datetime(np.array(xr.open_mfdataset(fp+'U').isel(time = slice(0,25)).time))

var_data = dict(
    features = (['time', 'lon', 'lat', 'n_channel'], data),
    labels_2d = (['time', 'lon', 'lat'], Y)
)

coords = dict(
    n_channel = (['n_channel'], np.array(['IWV', 'EFLUX', 'LWTNET', 'SF', 'SLP', 'T', 'U', 'V'])),
    time = (['time'], variable_times),   
)

ds = xr.Dataset(
    data_vars = var_data, 
    coords = coords
)

ds = ds.fillna(0)
ds


<xarray.Dataset>
Dimensions:    (time: 25, lon: 256, lat: 32, n_channel: 8)
Coordinates:
  * n_channel  (n_channel) <U6 'IWV' 'EFLUX' 'LWTNET' 'SF' 'SLP' 'T' 'U' 'V'
  * time       (time) datetime64[ns] 1980-01-01 ... 1980-01-07
Dimensions without coordinates: lon, lat
Data variables:
    features   (time, lon, lat, n_channel) float64 0.4602 -1.126 0.0 ... 0.0 0.0
    labels_2d  (time, lon, lat) int64 0 0 0 0 0 0 0 0 0 0 ... 0 0 0 0 0 0 0 0 0

In [76]:
## get even number of timesteps with/without ARs
AR_index = np.squeeze(np.where(ds.labels_2d.max(dim = ('lat','lon')).load()==1))
noAR_index = np.setdiff1d(np.arange(len(ds.time)), AR_index)
np.random.shuffle(noAR_index)
noAR_index = noAR_index[0:len(AR_index)]
select = np.sort(np.concatenate((noAR_index, AR_index)))
data = ds.isel(time = select)



In [77]:
data

<xarray.Dataset>
Dimensions:    (time: 6, lon: 256, lat: 32, n_channel: 8)
Coordinates:
  * n_channel  (n_channel) <U6 'IWV' 'EFLUX' 'LWTNET' 'SF' 'SLP' 'T' 'U' 'V'
  * time       (time) datetime64[ns] 1980-01-02T06:00:00 ... 1980-01-07
Dimensions without coordinates: lon, lat
Data variables:
    features   (time, lon, lat, n_channel) float64 0.4091 -0.5941 ... 0.0 0.0
    labels_2d  (time, lon, lat) int64 0 0 0 0 0 0 0 0 0 0 ... 0 0 0 0 0 0 0 0 0

In [78]:
# split into training, validating, and testing
index = np.arange(len(data.time))
split1, split2 = int(.7*len(index)), int(.85*len(index))

np.random.shuffle(index)
index_train, index_validate, index_test = index[:split1], index[split1:split2], index[split2:]
index_train.sort()
index_validate.sort()
index_test.sort()

ds_train = data.isel(time = index_train)
ds_test = data.isel(time = index_test)
ds_validate = data.isel(time = index_validate)

# ds_train.to_netcdf('/rc_scratch/reba1583/train.nc')
# ds_test.to_netcdf('/rc_scratch/reba1583/test.nc')
# ds_validate.to_netcdf('/rc_scratch/reba1583/validate.nc')



In [82]:
ds_validate

<xarray.Dataset>
Dimensions:    (time: 1, lon: 256, lat: 32, n_channel: 8)
Coordinates:
  * n_channel  (n_channel) <U6 'IWV' 'EFLUX' 'LWTNET' 'SF' 'SLP' 'T' 'U' 'V'
  * time       (time) datetime64[ns] 1980-01-07
Dimensions without coordinates: lon, lat
Data variables:
    features   (time, lon, lat, n_channel) float64 1.246 -0.3216 0.0 ... 0.0 0.0
    labels_2d  (time, lon, lat) int64 0 0 0 0 0 0 0 0 0 0 ... 0 0 0 0 0 0 0 0 0

In [ ]:
# may have to turn this into a script 

In [ ]:

# X data
fp = '/rc_scratch/reba1583/variable_yr_files_4/'

IWV = xr.open_mfdataset(fp+'IWV').IWV.values
EFLUX = xr.open_mfdataset(fp+'EFLUX').EFLUX.values
LWTNET = xr.open_mfdataset(fp+'LWTNET').LWTNET.values
SF = xr.open_mfdataset(fp+'sf').sf.values
SLP = xr.open_mfdataset(fp+'SLP').SLP.values
T = xr.open_mfdataset(fp+'T').T.values
U = xr.open_mfdataset(fp+'U').U.values
V = xr.open_mfdataset(fp+'V').V.values

In [1]:
data = np.stack([ IWV, EFLUX, LWTNET,
                 SF, SLP, T, U, V])



NameError: name 'np' is not defined

In [ ]:
del V 
del U
del T
del IWV
del EFLUX
del SF
del SLP
del LWTNET

In [2]:
# Y data
Y = xr.open_mfdataset('/pl/active/ATOC_SynopticMet/data/ar_data/Research3/Data/Y/*')
Y = Y.Y.values

<xarray.Dataset>
Dimensions:  (time: 58440, lat_index: 32, lon: 256)
Coordinates:
  * time     (time) datetime64[ns] 1980-01-01 ... 2019-12-31T18:00:00
    lat      (lat_index) float64 dask.array<chunksize=(32,), meta=np.ndarray>
  * lon      (lon) float64 -180.0 -178.6 -177.2 -175.8 ... 177.2 178.6 180.0
Dimensions without coordinates: lat_index
Data variables:
    Y        (time, lat_index, lon) float64 dask.array<chunksize=(1464, 32, 256), meta=np.ndarray>

In [ ]:
# times for final xarray
variable_times = pd.to_datetime(np.array(xr.open_mfdataset(fp+'U').time))

In [ ]:
var_data = dict(
    features = (['n_channel', 'time', 'lat', 'lon' ], data),
    labels_2d = (['time', 'lat', 'lon'], Y)
)

coords = dict(
    n_channel = (['n_channel'], np.array(['IWV', 'EFLUX', 'LWTNET', 'SF', 'SLP', 'T', 'U', 'V'])),
    time = (['time'], variable_times),
    
)


In [ ]:
ds = xr.Dataset(
    data_vars = var_data, 
    coords = coords
)


In [ ]:
# check shape here - compare to 4day testing I think you need to rearrange them

In [ ]:
## get even number of timesteps with/without ARs
AR_index = np.squeeze(np.where(ds.Y.max(dim = ('lat','lon')).load()==1))
noAR_index = np.setdiff1d(np.arange(len(ds.time)), AR_index)
np.random.shuffle(noAR_index)
noAR_index = noAR_index[0:len(AR_index)]
select = np.sort(np.concatenate((noAR_index, AR_index)))
data = ds.isel(time = select)




In [ ]:
# split into training, validating, and testing
index = np.arange(len(y_data.time))
split1, split2 = int(.7*len(index)), int(.85*len(index))

np.random.shuffle(index)
index_train, index_validate, index_test = index[:split1], index[split1:split2], index[split2:]
index_train.sort()
index_validate.sort()
index_test.sort()

ds_train = data.isel(time = index_train)
ds_test = data.isel(time = index_test)
ds_validate = data.isel(time = index_validate)



In [ ]:
ds_train.to_netcdf('rc_scratch/reba1583/train')
ds_test.to_netcdf('rc_scratch/reba1583/test')
ds_validate.to_netcdf('rc_scratch/reba1583/validate')

